<a href="https://colab.research.google.com/github/yamatai-wizard/AI_Code_Classification_Competition/blob/1/AI%E7%94%9F%E6%88%90%E3%82%B3%E3%83%BC%E3%83%89%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving submission.zip to submission.zip


In [ ]:
import zipfile
import io

zip_file_path = 'submission.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('submission_dir')

In [ ]:
import json
import pandas as pd

def load_jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# train.jsonl の読み込み
train_df = load_jsonl_to_dataframe('submission_dir/submission/train.jsonl')
print("Train DataFrame Head:")
print(train_df.head())

# test.jsonl の読み込み
test_df = load_jsonl_to_dataframe('submission_dir/submission/test.jsonl')
print("\nTest DataFrame Head:")
print(test_df.head())

prediction_df = load_jsonl_to_dataframe('submission_dir/submission/prediction.txt.sample')
print("\nPrediction DataFrame Head:")
print(prediction_df.head())

Train DataFrame Head:
   id  is_generated language  \
0   1          True        C   
1   2          True  Python3   
2   3          True     Java   
3   4          True        C   
4   5         False  Python3   

                                                code  
0  #include <stdio.h>\n#include <stdlib.h>\n#incl...  
1  import sys\n\n# Define mappings as constants f...  
2  public class AlphaNumberProcessor {\n  public ...  
3  #include <stdio.h>\n#include <string.h>\n\nint...  
4  import sys\n\ndef decimal_to_alpha(number):\n ...  

Test DataFrame Head:
    id language                                               code
0  285  Python3  import sys\nimport argparse\n\ndef is_ascii(s)...
1  286  Python3  import sys\nimport json\n\ndef predict_next_ro...
2  287     Java  package track;\n\npublic class App {\n  public...
3  288      C++  #include <iostream>\n#include <queue>\n#includ...
4  289      PHP  <?php\n$command = $argv[1];\n$input_file = $ar...

Prediction DataFrame Head:
   

In [ ]:
train_df

,id,is_generated,language,code
0,1,True,C,#include <stdio.h>\n#include <stdlib.h>\n#incl...
1,2,True,Python3,import sys\n\n# Define mappings as constants f...
2,3,True,Java,public class AlphaNumberProcessor {\n public ...
3,4,True,C,#include <stdio.h>\n#include <string.h>\n\nint...
4,5,False,Python3,import sys\n\ndef decimal_to_alpha(number):\n ...
...,...,...,...,...
279,280,True,C++,#include <iostream>\n#include <string>\n#inclu...
280,281,False,Python3,import sys\n\n\ndef decode(s):\n return sum...
281,282,False,Python3,import sys\n\ndef getInfo(argv):\n func_type ...
282,283,False,PHP,<?php namespace Track;\n\nfunction decode($alp...


In [ ]:
test_df

,id,language,code
0,285,Python3,import sys\nimport argparse\n\ndef is_ascii(s)...
1,286,Python3,import sys\nimport json\n\ndef predict_next_ro...
2,287,Java,package track;\n\npublic class App {\n public...
3,288,C++,#include <iostream>\n#include <queue>\n#includ...
4,289,PHP,<?php\n$command = $argv[1];\n$input_file = $ar...
...,...,...,...
2648,2933,PHP,"<?php namespace Track;\nini_set(""memory_limit""..."
2649,2934,Java,import java.nio.file.Files;\nimport java.nio.f...
2650,2935,Java,package track;\n\nimport java.util.Scanner;\ni...
2651,2936,C,#include <stdio.h>\n#include <stdlib.h>\n#incl...


In [ ]:
prediction_df

,0
0,0
1,1
2,0
3,1
4,0
...,...
2648,0
2649,1
2650,0
2651,1


In [ ]:
# 必要なライブラリをインポート
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# 前回のコードで読み込んだ DataFrame を使います
# 例: train_df, test_df

# 簡単な特徴量（コードの文字数と行数）を作成
train_df['code_length'] = train_df['code'].apply(len)
test_df['code_length'] = test_df['code'].apply(len)

train_df['code_lines'] = train_df['code'].apply(lambda x: x.count('\n'))
test_df['code_lines'] = test_df['code'].apply(lambda x: x.count('\n'))

# ラベル（is_generated）を数値に変換
train_df['is_generated_int'] = train_df['is_generated'].astype(int)

# 訓練データと検証データに分割
X = train_df[['code_length', 'code_lines', 'code']] # 'code'は後で使うので含めておく
y = train_df['is_generated_int']

# train_test_split でデータを分割（テストデータの比率を20%に設定）
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# TF-IDFベクトライザのインスタンス化
tfidf_vectorizer = TfidfVectorizer(max_features=10000, # 使う単語（トークン）の数を制限
                                   ngram_range=(1, 2),  # 1-gramと2-gramを使用
                                   min_df=5)            # 5文書未満に出現する単語は無視

# 訓練データのコードでベクトライザを学習し、特徴量行列に変換
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['code'])
# 検証データにも同じベクトライザを適用して特徴量行列に変換
X_val_tfidf = tfidf_vectorizer.transform(X_val['code'])
# テストデータにも適用
X_test_tfidf = tfidf_vectorizer.transform(test_df['code'])

In [ ]:
import lightgbm as lgb
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# 簡単な特徴量を疎行列に変換
X_train_meta = X_train[['code_length', 'code_lines']].values
X_val_meta = X_val[['code_length', 'code_lines']].values
X_test_meta = test_df[['code_length', 'code_lines']].values

# TF-IDF特徴量と簡単な特徴量を水平方向に結合
X_train_combined = hstack([X_train_tfidf, X_train_meta])
X_val_combined = hstack([X_val_tfidf, X_val_meta])

# LGBMモデルのインスタンス化
lgbm_model = lgb.LGBMClassifier(objective='binary', random_state=42)

# モデルの訓練
lgbm_model.fit(X_train_combined, y_train)

[LightGBM] [Info] Number of positive: 117, number of negative: 110
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000926 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4598
[LightGBM] [Info] Number of data points in the train set: 227, number of used features: 253
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.515419 -> initscore=0.061694
[LightGBM] [Info] Start training from score 0.061694
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

LGBMClassifier(objective='binary', random_state=42)

In [ ]:
# 検証データでの予測
y_train_pred_proba = lgbm_model.predict_proba(X_train_combined)[:, 1] # 確率を取得
y_train_pred = (y_train_pred_proba > 0.5).astype(int)              # 0.5を閾値としてクラスを予測
f1_score(y_train, y_train_pred)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


1.0

In [ ]:
# 検証データでの予測
y_val_pred_proba = lgbm_model.predict_proba(X_val_combined)[:, 1] # 確率を取得
y_val_pred = (y_val_pred_proba > 0.5).astype(int)              # 0.5を閾値としてクラスを予測

# 評価指標の計算
accuracy = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_pred_proba)

print(f"Accuracy on validation set: {accuracy:.4f}")
print(f"F1 Score on validation set: {f1:.4f}")
print(f"ROC AUC on validation set: {roc_auc:.4f}")

Accuracy on validation set: 0.9298
F1 Score on validation set: 0.9355
ROC AUC on validation set: 0.9865


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


In [ ]:
# テストデータでの予測
X_test_combined = hstack([X_test_tfidf, X_test_meta])
y_test_pred_proba = lgbm_model.predict_proba(X_test_combined)[:, 1]

# 確率値からクラスラベル（0または1）に変換
y_test_pred_binary = (y_test_pred_proba > 0.5).astype(int)

# DataFrameではなく、直接予測結果をリストとして用意
predictions = y_test_pred_binary.tolist()

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


In [ ]:
import os

# 提出用フォルダのパス
output_dir = 'submission'
os.makedirs(output_dir, exist_ok=True)

# 予測結果をprediction.txtに書き出し
with open(os.path.join(output_dir, 'prediction.txt'), 'w') as f:
    for pred in predictions:
        f.write(str(pred) + '\n')

print(f"'{os.path.join(output_dir, 'prediction.txt')}' を作成しました。")

'submission/prediction.txt' を作成しました。


In [ ]:
# ダウンロードするファイルのパス
file_to_download = 'submission/prediction.txt'

# ファイルをダウンロード
files.download(file_to_download)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'submission/prediction.txt' のダウンロードを開始しました。


スコアは0.6872502724300763